# Source File: https://currents.soest.hawaii.edu/ocn_data_analysis/_static/Spectrum.html

In [ ]:
%matplotlib inline 
# substitude notebook for inline above to get interactive
# inline plots

import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as ss

from pycurrents.num import spectra

plt.rcParams['figure.dpi'] = 90

In [ ]:
def datafaker(nt, dt=1, freqs=None, color='w',
              amp=1, 
              complex=True,
              repeatable=True):
    """
    Generate fake data with optional sinusoids (all the
    same amplitude) and with red, white, or blue noise
    of arbitrary amplitude.
    
    *nt* : number of points
    *dt* : time increment in arbitrary time units
    *freqs* : None, or a sequence of frequencies in
        cycles per unit time. 
    *color* : 'r', 'w', 'b'
    *amp* : amplitude of red, white, or blue noise
    *complex* : True, False
    *repeatable* : True, False

    Returns t, x
    """
    if repeatable:
        np.random.seed(1)    
    noise = np.random.randn(nt + 1) + 1j * np.random.randn(nt + 1)
    
    if color == 'r':
        noise = np.cumsum(noise) / 10 
        noise -= noise.mean()
    elif color == 'b':
        noise = np.diff(noise)
    noise = noise[:nt]
    x = amp * noise

    t = np.arange(nt, dtype=float) * dt
    
    for f in freqs:
        sinusoid = np.exp(2 * np.pi * 1j * f * t)
        x += sinusoid
    if not complex:
        x = np.real(x)
        
    return t, x

In [ ]:
nt = 240

dt = 1/24 # 1 hour sample interval
tides = [24/12.42, 24/12]

t, h = datafaker(nt, dt=dt, freqs=tides, amp=1, 
                 color='r',
                 complex=False)
fig, ax = plt.subplots()
ax.plot(t, h)
ax.set_xlabel('days');

In [ ]:
def spectrum1(h, dt=1):
    """
    First cut at spectral estimation: very crude.
    
    Returns frequencies, power spectrum, and
    power spectral density.
    Only positive frequencies between (and not including)
    zero and the Nyquist are output.
    """
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.fft.fftfreq(nt, d=dt)[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = np.abs(ft) ** 2
    # Double to account for the energy in the negative frequencies.
    psraw *= 2
    # Normalization for Power Spectrum
    psraw /= nt**2
    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw

In [ ]:
# Pick two data lengths as number of samples.  These will be used
# throughout the following examples.  They need to be quite large
# for the examples to work well.
n1 = 2400
n2 = 24000

dfkw = dict(dt=dt, freqs=tides, amp=1, color='r', complex=False)

t, h1 = datafaker(n1, **dfkw)
freqs1, ps1, psd1 = spectrum1(h1, dt=dt)

t, h2 = datafaker(n2, **dfkw)
freqs2, ps2, psd2 = spectrum1(h2, dt=dt)

fig, axs = plt.subplots(ncols=2, sharex=True)
axs[0].loglog(freqs1, psd1, 'r',
              freqs2, psd2, 'b', alpha=0.5)
axs[1].loglog(freqs1, ps1, 'r', 
              freqs2, ps2, 'b', alpha=0.5)
axs[0].set_title('Power Spectral Density')
axs[1].set_title('Power Spectrum')
axs[1].axis('tight', which='x');

In [ ]:
print('PS sum:   %.2f, %.2f' % (ps1.sum(), ps2.sum()))
print('Variance: %.2f, %.2f' % (h1.var(), h2.var()))
print('Differences: %g, %g' % (h1.var() - ps1.sum(),
                               h2.var() - ps2.sum()))

In [ ]:
print('Tidal peak')

# Find a small band centered on the tidal constituents:
cond1 = (freqs1 < 24/11) & (freqs1 > 24/13.5) 
df1 = 1 / (len(h1) * dt)
psdint1 = psd1[cond1].sum() * df1

cond2 = (freqs2 < 24/11.9) & (freqs2 > 24/12.52) 
df2 = 1 / (len(h2) * dt)
psdint2 = psd2[cond2].sum() * df2

print('PSD integral:%.2f, %.2f' % (psdint1, psdint2))

print('PS tidal max: %.2g, %.2g' % (ps1[cond1].max(),
                                    ps2[cond2].max()) )
print('PSD tidal max: %.2g, %.2g' % (psd1[cond1].max(),
                                    psd2[cond2].max()) )

In [ ]:
def spectrum2(h, dt=1, nsmooth=5):
    """
    Add simple boxcar smoothing to the raw periodogram.
    
    Chop off the ends to avoid end effects.
    """
    freqs, ps, psd = spectrum1(h, dt=dt)
    weights = np.ones(nsmooth, dtype=float) / nsmooth
    ps_s = np.convolve(ps, weights, mode='valid')
    psd_s = np.convolve(psd, weights, mode='valid')
    freqs_s = np.convolve(freqs, weights, mode='valid')
    return freqs_s, ps_s, psd_s

In [ ]:
dfkw = dict(dt=dt, freqs=tides, amp=1, color='r', complex=False)

t, h1 = datafaker(n1, **dfkw)
freqs1, ps1, psd1 = spectrum2(h1, dt=dt)

t, h2 = datafaker(n2, **dfkw)
freqs2, ps2, psd2 = spectrum2(h2, dt=dt)

fig, axs = plt.subplots(ncols=2, sharex=True)
axs[0].loglog(freqs1, psd1, 'r',
              freqs2, psd2, 'b', alpha=0.5)
axs[1].loglog(freqs1, ps1, 'r', 
              freqs2, ps2, 'b', alpha=0.5)
axs[0].set_title('Power Spectral Density')
axs[1].set_title('Power Spectrum')
axs[1].axis('tight', which='x');

In [ ]:
def spectrum3(h, dt=1, nsmooth=5):
    """
    Detrend first.
    """
    t = np.arange(len(h))
    p = np.polyfit(t, h, 1)
    h_detrended = h - np.polyval(p, t)
    return spectrum2(h_detrended, dt=dt, nsmooth=nsmooth)
    

In [ ]:
dfkw = dict(dt=dt, freqs=tides, amp=1, color='r', complex=False)
# smooth more heavily to make plot less noisy
ns = 13

t, h1 = datafaker(n1, **dfkw)
t, h2 = datafaker(n2, **dfkw)

freqs1, ps1, psd1 = spectrum3(h1, dt=dt, nsmooth=ns)
freqs2, ps2, psd2 = spectrum3(h2, dt=dt, nsmooth=ns)
freqs2a, ps2a, psd2a = spectrum2(h2, dt=dt, nsmooth=ns)

fig, axs = plt.subplots(ncols=2, sharex=True)

axs[0].loglog(freqs2, psd2, 'r', alpha=0.5, label='detrended') 
axs[0].loglog(freqs2, psd2a, 'k', alpha=0.5, label='raw')
axs[0].set_title('PSD')
axs[0].legend(loc='lower left')

axs[1].loglog(freqs2, psd2a/psd2, 'r')
axs[1].set_title('PSD raw/detrended')
axs[1].axis('tight', which='x');


In [ ]:
def detrend(h):
    n = len(h)
    t = np.arange(n)
    p = np.polyfit(t, h, 1)
    h_detrended = h - np.polyval(p, t)
    return h_detrended
    
def quadwin(n):
    """
    Quadratic (or "Welch") window
    """
    t = np.arange(n)
    win = 1 - ((t - 0.5 * n) / (0.5 * n)) ** 2
    return win

def spectrum4(h, dt=1, nsmooth=5):
    """
    Detrend and apply a quadratic window.
    """
    n = len(h)

    h_detrended = detrend(h)
    
    winweights = quadwin(n)
    h_win = h_detrended * winweights
    
    freqs, ps, psd = spectrum2(h_win, dt=dt, nsmooth=nsmooth)
    
    # Compensate for the energy suppressed by the window.
    psd *= n / (winweights**2).sum()
    ps *= n**2 / winweights.sum()**2
    
    return freqs, ps, psd



In [ ]:
dfkw = dict(dt=dt, freqs=tides, amp=1, color='r', complex=False)

t1, h1 = datafaker(n1, **dfkw)
#t2, h2 = datafaker(n2, **dfkw)

In [ ]:
def win_dtr(h):
    n = len(h)
    h_detrended = detrend(h)    
    winweights = quadwin(n)
    h_win = h_detrended * winweights
    return h_win

fig, ax = plt.subplots()
ax.plot(t1, h1, 'r', t1, win_dtr(h1), 'b', alpha=0.5);

In [ ]:
freqs1, ps1, psd1 = spectrum4(h1, dt=dt)
freqs1a, ps1a, psd1a = spectrum3(h1, dt=dt)

fig, axs = plt.subplots(ncols=2)
axs[0].loglog(freqs1, psd1a, 'r', alpha=0.5, label='no window')
axs[0].loglog(freqs1, psd1, 'b', alpha=0.5, label='window')
axs[0].axis('tight', which='x')
axs[0].legend(loc='lower left')

i0, i1 = np.searchsorted(freqs1, [1.2, 3])
sl = slice(i0, i1)
axs[1].semilogy(freqs1[sl], psd1a[sl], 'r', 
              freqs1[sl], psd1[sl], 'b', alpha=0.5)
axs[1].axis('tight', which='x')
axs[0].set_title('Power Spectral Density')
axs[1].set_title('Power Spectral Density');

In [ ]:
df = 10 # DOF for 5-point boxcar smoother

# location of confidence limit bar
conf_x = 3
conf_y0 = 1

conf = conf_y0 * df / ss.chi2.ppf([0.025, 0.975], df)

fig, ax = plt.subplots()
ax.semilogy(freqs1, psd1, 'b', alpha=0.5)
ax.semilogy(freqs1a, psd1a, 'r', alpha=0.5)

ax.plot([conf_x, conf_x], conf, color='k', lw=1.5)
ax.plot(conf_x, conf_y0, color='k', linestyle='none', 
        marker='_', ms=8, mew=2)

ax.set_xlim(1, 5)
ax.set_title('Power Spectral Density')
ax.set_xlabel('Cycles per day');


In [ ]:
# Use higher amplitude background noise, and shorter time 
# series for this illustration.

dfkw = dict(dt=dt, freqs=tides, amp=5, color='r', complex=False)
# smooth more heavily to make plot less noisy
ns = 13
n1, n2 = 480, 4800
t, h1 = datafaker(n1, **dfkw)
t, h2 = datafaker(n2, **dfkw)

freqs1, ps1, psd1 = spectrum4(h1, dt=dt, nsmooth=ns)
freqs2, ps2, psd2 = spectrum4(h2, dt=dt, nsmooth=ns)

fig, ax = plt.subplots()

ax.semilogx(freqs1, psd1 * freqs1, 'r', alpha=0.5)
ax.semilogx(freqs2, psd2 * freqs2, 'b', alpha=0.5)

ax.set_title('Variance-preserving plot example')
ax.set_xlabel('Cycles per day')
ax.set_ylabel('Variance per CPD');

In [ ]:
nt = 60

dt = 1/24 # 1 hour sample interval
tides = [24/12.42, 24/12]

t, uc = datafaker(nt, dt=dt, freqs=tides, amp=0.1, 
                 color='w',
                 complex=True)
fig, ax = plt.subplots()
ax.plot(t, uc.real, 'r', label='U')
ax.plot(t, uc.imag, 'b', label='V')
ax.legend(loc='upper right')
ax.set_xlabel('Days');

In [ ]:
nt = 120

dt = 1/24 # 1 hour sample interval
tides = [24/12.42, 24/12, -1]

t, uc = datafaker(nt, dt=dt, freqs=tides, amp=0.5, 
                 color='w',
                 complex=True)
fig, ax = plt.subplots()
ax.plot(t, uc.real, 'r', label='U')
ax.plot(t, uc.imag, 'b', label='V')
ax.legend(loc='upper right')
ax.set_xlabel('Days');

In [ ]:
s = spectra.spectrum(uc, nfft=None, dt=1/24, 
                     window='quadratic',
                     smooth=3)

In [ ]:
fig, ax = plt.subplots()
ax.plot(s.ccwfreqs, s.ccwpsd, 'r', label='CCW')
ax.plot(s.cwfreqs, s.cwpsd, 'b', label='CW')
ax.legend(loc='upper right')
ax.set_xlabel('Cycles per day')
ax.set_ylabel('Something squared per cycle per day');